In [1]:
import numpy as np
import exafmm.modified_helmholtz as mod_helm

In [2]:
mod_helm.__doc__

"exafmm's submodule for Modified Helmholtz kernel"

### 1. create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

# generate random positions for particles
src_coords = np.random.random((nsrcs, 3))
trg_coords = np.random.random((nsrcs, 3))

# generate random charges for sources
src_charges = np.random.random(nsrcs)

In [4]:
# create a list of source instances
sources = mod_helm.init_sources(src_coords, src_charges)

# create a list of target instances
targets = mod_helm.init_targets(trg_coords)

### 2. create a ModifiedHelmholtzFmm instance

In [5]:
p = 10
ncrit = 400
depth = 4
wavek = 10
fmm = mod_helm.ModifiedHelmholtzFmm(p=10, ncrit=400, depth=4, wavek=7.5, filename='modhelm_example.dat')

### 3. setup fmm

Given sources, targets and FMM configurations, `setup()` builds the tree and interaction list and pre-compute invariant matrices.

In [6]:
tree = mod_helm.setup(sources, targets, fmm)

### 4. evaluate

`evaluate()` triggers the evaluation and returns a $n_{trg} \times 4$ `numpy.ndarray`.
The $i$-th row of `trg_values` starts with the potential value of the $i$-th target, followed by its three gradient values.

In [7]:
trg_values = mod_helm.evaluate(tree, fmm)

In [8]:
trg_values[6]

array([  272.2419057 ,   727.77247052,  1358.14202039, -1650.88659291])

### 5. check accuracy (optional)

`fmm.verify(tree.leafs)` returns L2-norm the relative error of potential and gradient in a list. It only works when `skip_P2P` is set to `False`.

In [9]:
fmm.verify(tree.leafs)

[3.6296201820544796e-10, 5.997876824703602e-08]

### 6. update charges of sources and run FMM iteratively

In [10]:
niters = 4

for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    
    src_charges = np.random.random(nsrcs)         # generate new random charges          
    mod_helm.update_charges(tree, src_charges)      # update charges
    mod_helm.clear_values(tree)                     # clear values
    trg_values = mod_helm.evaluate(tree, fmm)       # evaluate potentials

    print("Error: ", fmm.verify(tree.leafs))       # check accuracy

---------- iteration 0 ----------
Error:  [2.9240677371073643e-10, 4.582230373984346e-08]
---------- iteration 1 ----------
Error:  [3.595824740268661e-10, 6.191079184722706e-08]
---------- iteration 2 ----------
Error:  [2.8681826732817513e-10, 4.914957353387267e-08]
---------- iteration 3 ----------
Error:  [3.134687905276501e-10, 5.843474500237767e-08]
